## Import Packages

In [ ]:
import pandas as pd
import numpy as np
import os
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns
from tqdm.notebook import tqdm
import tqdm.notebook as tq
import cv2

## Read Data

In [ ]:
%cd "/kaggle/input"
train = pd.read_csv("../input/smartathon/dataset/train.csv")
test = pd.read_csv("../input/smartathon/dataset/test.csv")
submission = pd.read_csv("../input/smartathon/dataset/sample_submission.csv")

## Reduce Size by 50% to get proper bounding boxes

In [ ]:
os.makedirs('/kaggle/working/smartathon/resize_images', exist_ok = True)
for i,j in enumerate(train['image_path']):
    img = cv2.imread("../input/smartathon/dataset/images/" + j, cv2.IMREAD_UNCHANGED) 
    width = 960
    height = 540 # keep original height
    dim = (width, height)
    # resize image
    resized = cv2.resize(img, dim)
    cv2.imwrite("/kaggle/working/smartathon/resize_images/" + j,resized)

## Plot Bounding Box images

In [ ]:
import cv2

def plot_one_box(x, img, color=None, label=None, line_thickness=None, Inverted=False):
    # Plots one bounding box on image img
    tl = line_thickness or 2 # line thickness
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl)
    if label:
        tf = tl # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 2, thickness=tf)[0]
    if Inverted == True:
        c1 = c2
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
    else:
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1) # filled
        cv2.putText(
        img,
        label,
        (c1[0], c1[1] - 2),
        0,
        tl / 2,
        [225, 255, 255],
        thickness=tf,
        lineType=cv2.LINE_AA,
        )

In [ ]:
# # Using readlines()
# file1 = open('../input/smartathon/dataset/train.csv', 'r')
# Lines = file1.readlines()

# count = 0
# # Strips the newline character
# for line in Lines:
#     #print(line)
#     if count == 0:
#         count += 1
#         continue
# print(count)
# file_id_path = line.split(',')[1]
# print(file_id_path)
# # open image in cv2
# img = cv2.imread("../input/smartathon/dataset/images/" + file_id_path)
# #print(img)
# h, w, c = img.shape
# cat = line.split(',')[2]
# xmax = int(float(line.split(',')[3])) * 2
# xmin = int(float(line.split(',')[4])) * 2
# ymax = int(float(line.split(',')[5])) * 2
# ymin = int(float(line.split(',')[6])) * 2
# # plot the box
# plot_one_box([xmin, ymin, xmax, ymax], img, color=(0, 255, 0), label=cat, line_thickness=2)
# # save the image
# # you might need to create the folder "drawn" first!
# cv2.imwrite("../working/" + file_id_path, img)
# print("Line {}: {}".format(count, line.strip()))
# count += 1

In [ ]:
## Keeping contant height and width
train['height'] = 540
train['width'] = 960

## Fixing issues with bounding boxes to keep xmax to 960 and xmin to 540 and keeping negative values to zero

In [ ]:
### fixing bounding box problems

for i,j in enumerate(train['xmax']):
    ##### xmax ---> 0 <= xmax <= 960
    if  j > 960:
        train['xmax'].iloc[i] = 960
    elif j < 0:
        train['xmax'].iloc[i] = 960
    
    #### xmin ----> 0 <= xmin <= xmax
    
    if train['xmin'].iloc[i] < 0 :
        train['xmin'].iloc[i] =0
    elif train['xmin'].iloc[i] > train['xmax'].iloc[i]:
        print("####### highlighting index xmin <= xmax ####" + str(i))
    
    ######## ymax ---> 0<= yamx <= 540
    
    if train['ymax'].iloc[i] > 540 :
        train['ymax'].iloc[i] =540
    elif train['ymax'].iloc[i] < 0:
        train['ymax'].iloc[i] = 540
    
    ######## ymin ---> 0<= ymin <= ymax
    if train['ymin'].iloc[i] < 0 :
        train['ymin'].iloc[i] =0
    elif train['ymin'].iloc[i] > train['ymax'].iloc[i]:
        print("####### highlighting index ymin <= ymax ####" + str(i))

In [ ]:
train = train.rename(columns={"xmax":"x_max","xmin":"x_min","ymax":"y_max","ymin":"y_min","class":"class_id","name":"class_name"})

In [ ]:
train.shape

## Normalizing Input data

In [ ]:
train['x_min'] = train.apply(lambda row: (row.x_min)/row.width, axis =1)
train['y_min'] = train.apply(lambda row: (row.y_min)/row.height, axis =1)

train['x_max'] = train.apply(lambda row: (row.x_max)/row.width, axis =1)
train['y_max'] = train.apply(lambda row: (row.y_max)/row.height, axis =1)

train['x_mid'] = train.apply(lambda row: (row.x_max+row.x_min)/2, axis =1)
train['y_mid'] = train.apply(lambda row: (row.y_max+row.y_min)/2, axis =1)

train['w'] = train.apply(lambda row: (row.x_max-row.x_min), axis =1)
train['h'] = train.apply(lambda row: (row.y_max-row.y_min), axis =1)

train['area'] = train['w']*train['h']
train.head()

In [ ]:
class_ids, class_names = list(zip(*set(zip(train.class_id, train.class_name))))
classes = list(np.array(class_names)[np.argsort(class_ids)])
classes = list(map(lambda x: str(x), classes))
classes

In [ ]:
### Create path for label text files
os.makedirs('/kaggle/working/smartathon/label_text_file', exist_ok = True)

### Stratified Split custom logic

* First split the lowest imabalnce class into 80:20 ratio
* Check images into training set
* Split next highly imbalance class and repeat the process

In [ ]:
train_idx = []
test_idx = []
for class_ in reversed(list(train.class_name.value_counts().index)):
    print(class_)
    df_class = train[(train.class_name == class_) 
                     & ~(train.image_path.isin(train[train.index.isin(train_idx)].image_path))
                     & ~(train.image_path.isin(train[train.index.isin(test_idx)].image_path))]
    
    if df_class.shape[0] == 1:
        # all data goes in train class
        train_idx.extend(list(df_class.index))
    else:
        print("Inside else ="+str(df_class.shape))
        # identify fractaion on the fly 
        total_rows_class = train[(train.class_name == class_)].shape[0]
        should_be = round(total_rows_class*0.8)
        # how many more to add 
        current_train_set_img = train[(train.index.isin(train_idx)) ].image_path.values
        already_present = train[(train.image_path.isin(current_train_set_img)) & (train.class_name == class_)].shape[0]
        print("should_be","already_present")
        print(should_be,already_present)
        sample_to_be = should_be-already_present
        if sample_to_be <= 0:
            sample_to_be = 0
        train_set_idx = list(df_class.sample(sample_to_be,random_state=42).index)
        test_set_idx = df_class[~(df_class.image_path.isin(df_class[df_class.index.isin(train_set_idx)].image_path))].index
#         print(len(train_set_idx),len(test_set_idx))
#         print(len(train_idx),len(test_idx))
        train_idx.extend(train_set_idx)
        test_idx.extend(test_set_idx)
#         print(len(train_idx),len(test_idx))

In [ ]:
train_stratified = train[train.index.isin(train_idx)]
test_stratified = train[train.index.isin(test_idx)]
train_final = train[train.image_path.isin(train_stratified.image_path.unique())]
test_final = train[train.image_path.isin(test_stratified.image_path.unique())]

In [ ]:
train_files = train_final.copy(deep=True)
val_files = test_final.copy(deep=True)

In [ ]:
train_files['bbox']= train_files[['class_id','x_mid','y_mid','w','h']].values.tolist()
val_files['bbox']= val_files[['class_id','x_mid','y_mid','w','h']].values.tolist()

In [ ]:
train_df = train_files.groupby("image_path")['bbox'].apply(list).reset_index(name="bbox")
valid_df = val_files.groupby("image_path")['bbox'].apply(list).reset_index(name="bbox")

In [ ]:
train_df.shape

In [ ]:
valid_df.shape

In [ ]:
def process_images(df):
    for _, row in tq.tqdm(df.iterrows(), total=len(df)):
        yolo_data = row['bbox']
        yolo_data = np.array(yolo_data)
        image_id = row['image_path'].split('.')[0]
        np.savetxt(os.path.join('yolov5/base_dir', 
                    f"/kaggle/working/smartathon/label_text_file/{image_id}.txt"),
                    yolo_data, 
                    fmt=["%d", "%f", "%f", "%f", "%f"]
                    ) # fmt means format the columns

In [ ]:
process_images(train_df)
process_images(valid_df)

In [ ]:
train_df['image_path'] = "/kaggle/working/smartathon/resize_images/" + train_df['image_path']
valid_df['image_path'] = "/kaggle/working/smartathon/resize_images/" + valid_df['image_path']

In [ ]:
train_image_path = train_df['image_path']
validation_image_path = valid_df['image_path']

## Copying Files to create YOLOV5 folders

In [ ]:
os.makedirs('/kaggle/working/smartathon/labels/train', exist_ok = True)
os.makedirs('/kaggle/working/smartathon/labels/val', exist_ok = True)
os.makedirs('/kaggle/working/smartathon/images/train', exist_ok = True)
os.makedirs('/kaggle/working/smartathon/images/val', exist_ok = True)
label_dir = '/kaggle/working/smartathon/label_text_file'
for file in tqdm(train_image_path):
    shutil.copy(file, '/kaggle/working/smartathon/images/train')
    filename = file.split('/')[-1].split('.')[0]
    shutil.copy(os.path.join(label_dir, filename+'.txt'), '/kaggle/working/smartathon/labels/train')
    
for file in tqdm(validation_image_path):
    shutil.copy(file, '/kaggle/working/smartathon/images/val')
    filename = file.split('/')[-1].split('.')[0]
    shutil.copy(os.path.join(label_dir, filename+'.txt'), '/kaggle/working/smartathon/labels/val')

## YOLOv5 Modeling


In [ ]:
len(classes)

In [ ]:
from os import listdir
from os.path import isfile, join
import yaml

cwd = '/kaggle/working/'

with open(join( cwd , 'train.txt'), 'w') as f:
    for path in glob('/kaggle/working/smartathon/images/train/*'):
        f.write(path+'\n')
            
with open(join( cwd , 'val.txt'), 'w') as f:
    for path in glob('/kaggle/working/smartathon/images/val/*'):
        f.write(path+'\n')

data = dict(
    train =  join( cwd , 'train.txt') ,
    val   =  join( cwd , 'val.txt' ),
    nc    = 11,
    names = classes
    )

with open(join( cwd , 'smartathon.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

f = open(join( cwd , 'smartathon.yaml'), 'r')
print('\nyaml:')
print(f.read())

In [ ]:
# https://www.kaggle.com/ultralytics/yolov5
# !git clone https://github.com/ultralytics/yolov5  # clone repo
# %cd yolov5
shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5', '/kaggle/working/yolov5')
os.chdir('/kaggle/working/yolov5')
# %pip install -qr requirements.txt # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
!wandb login "31593110873a82f820efeeda6f8b0b5ec5b5d9d2"

## YOLOV5 - Changing the metrics (Look for model fitness function), we focused more on mAP@0.5 

In [ ]:
%%writefile /kaggle/working/yolov5/utils/metrics.py

# %load /kaggle/working/yolov5/utils/metrics.py
# Model validation metrics

from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import torch

from . import general


def fitness(x):
    # Model fitness as a weighted combination of metrics
    w = [0.0, 0.0, 0.9, 0.1]  # weights for [P, R, mAP@0.5, mAP@0.5:0.95]
    return (x[:, :4] * w).sum(1)


def ap_per_class(tp, conf, pred_cls, target_cls, plot=False, save_dir='precision-recall_curve.png', names=[]):
    """ Compute the average precision, given the recall and precision curves.
    Source: https://github.com/rafaelpadilla/Object-Detection-Metrics.
    # Arguments
        tp:  True positives (nparray, nx1 or nx10).
        conf:  Objectness value from 0-1 (nparray).
        pred_cls:  Predicted object classes (nparray).
        target_cls:  True object classes (nparray).
        plot:  Plot precision-recall curve at mAP@0.5
        save_dir:  Plot save directory
    # Returns
        The average precision as computed in py-faster-rcnn.
    """

    # Sort by objectness
    i = np.argsort(-conf)
    tp, conf, pred_cls = tp[i], conf[i], pred_cls[i]

    # Find unique classes
    unique_classes = np.unique(target_cls)

    # Create Precision-Recall curve and compute AP for each class
    px, py = np.linspace(0, 1, 1000), []  # for plotting
    pr_score = 0.1  # score to evaluate P and R https://github.com/ultralytics/yolov3/issues/898
    s = [unique_classes.shape[0], tp.shape[1]]  # number class, number iou thresholds (i.e. 10 for mAP0.5...0.95)
    ap, p, r = np.zeros(s), np.zeros(s), np.zeros(s)
    for ci, c in enumerate(unique_classes):
        i = pred_cls == c
        n_l = (target_cls == c).sum()  # number of labels
        n_p = i.sum()  # number of predictions

        if n_p == 0 or n_l == 0:
            continue
        else:
            # Accumulate FPs and TPs
            fpc = (1 - tp[i]).cumsum(0)
            tpc = tp[i].cumsum(0)

            # Recall
            recall = tpc / (n_l + 1e-16)  # recall curve
            r[ci] = np.interp(-pr_score, -conf[i], recall[:, 0])  # r at pr_score, negative x, xp because xp decreases

            # Precision
            precision = tpc / (tpc + fpc)  # precision curve
            p[ci] = np.interp(-pr_score, -conf[i], precision[:, 0])  # p at pr_score

            # AP from recall-precision curve
            for j in range(tp.shape[1]):
                ap[ci, j], mpre, mrec = compute_ap(recall[:, j], precision[:, j])
                if plot and (j == 0):
                    py.append(np.interp(px, mrec, mpre))  # precision at mAP@0.5

    # Compute F1 score (harmonic mean of precision and recall)
    f1 = 2 * p * r / (p + r + 1e-16)

    if plot:
        plot_pr_curve(px, py, ap, save_dir, names)

    return p, r, ap, f1, unique_classes.astype('int32')


def compute_ap(recall, precision):
    """ Compute the average precision, given the recall and precision curves
    # Arguments
        recall:    The recall curve (list)
        precision: The precision curve (list)
    # Returns
        Average precision, precision curve, recall curve
    """

    # Append sentinel values to beginning and end
    mrec = np.concatenate(([0.], recall, [recall[-1] + 0.01]))
    mpre = np.concatenate(([1.], precision, [0.]))

    # Compute the precision envelope
    mpre = np.flip(np.maximum.accumulate(np.flip(mpre)))

    # Integrate area under curve
    method = 'interp'  # methods: 'continuous', 'interp'
    if method == 'interp':
        x = np.linspace(0, 1, 101)  # 101-point interp (COCO)
        ap = np.trapz(np.interp(x, mrec, mpre), x)  # integrate
    else:  # 'continuous'
        i = np.where(mrec[1:] != mrec[:-1])[0]  # points where x axis (recall) changes
        ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])  # area under curve

    return ap, mpre, mrec


class ConfusionMatrix:
    # Updated version of https://github.com/kaanakan/object_detection_confusion_matrix
    def __init__(self, nc, conf=0.25, iou_thres=0.45):
        self.matrix = np.zeros((nc + 1, nc + 1))
        self.nc = nc  # number of classes
        self.conf = conf
        self.iou_thres = iou_thres

    def process_batch(self, detections, labels):
        """
        Return intersection-over-union (Jaccard index) of boxes.
        Both sets of boxes are expected to be in (x1, y1, x2, y2) format.
        Arguments:
            detections (Array[N, 6]), x1, y1, x2, y2, conf, class
            labels (Array[M, 5]), class, x1, y1, x2, y2
        Returns:
            None, updates confusion matrix accordingly
        """
        detections = detections[detections[:, 4] > self.conf]
        gt_classes = labels[:, 0].int()
        detection_classes = detections[:, 5].int()
        iou = general.box_iou(labels[:, 1:], detections[:, :4])

        x = torch.where(iou > self.iou_thres)
        if x[0].shape[0]:
            matches = torch.cat((torch.stack(x, 1), iou[x[0], x[1]][:, None]), 1).cpu().numpy()
            if x[0].shape[0] > 1:
                matches = matches[matches[:, 2].argsort()[::-1]]
                matches = matches[np.unique(matches[:, 1], return_index=True)[1]]
                matches = matches[matches[:, 2].argsort()[::-1]]
                matches = matches[np.unique(matches[:, 0], return_index=True)[1]]
        else:
            matches = np.zeros((0, 3))

        n = matches.shape[0] > 0
        m0, m1, _ = matches.transpose().astype(np.int16)
        for i, gc in enumerate(gt_classes):
            j = m0 == i
            if n and sum(j) == 1:
                self.matrix[gc, detection_classes[m1[j]]] += 1  # correct
            else:
                self.matrix[gc, self.nc] += 1  # background FP

        if n:
            for i, dc in enumerate(detection_classes):
                if not any(m1 == i):
                    self.matrix[self.nc, dc] += 1  # background FN

    def matrix(self):
        return self.matrix

    def plot(self, save_dir='', names=()):
        try:
            import seaborn as sn

            array = self.matrix / (self.matrix.sum(0).reshape(1, self.nc + 1) + 1E-6)  # normalize
            array[array < 0.005] = np.nan  # don't annotate (would appear as 0.00)

            fig = plt.figure(figsize=(12, 9), tight_layout=True)
            sn.set(font_scale=1.0 if self.nc < 50 else 0.8)  # for label size
            labels = (0 < len(names) < 99) and len(names) == self.nc  # apply names to ticklabels
            sn.heatmap(array, annot=self.nc < 30, annot_kws={"size": 8}, cmap='Blues', fmt='.2f', square=True,
                       xticklabels=names + ['background FN'] if labels else "auto",
                       yticklabels=names + ['background FP'] if labels else "auto").set_facecolor((1, 1, 1))
            fig.axes[0].set_xlabel('True')
            fig.axes[0].set_ylabel('Predicted')
            fig.savefig(Path(save_dir) / 'confusion_matrix.png', dpi=250)
        except Exception as e:
            pass

    def print(self):
        for i in range(self.nc + 1):
            print(' '.join(map(str, self.matrix[i])))


# Plots ----------------------------------------------------------------------------------------------------------------

def plot_pr_curve(px, py, ap, save_dir='.', names=()):
    fig, ax = plt.subplots(1, 1, figsize=(9, 6), tight_layout=True)
    py = np.stack(py, axis=1)

    if 0 < len(names) < 21:  # show mAP in legend if < 10 classes
        for i, y in enumerate(py.T):
            ax.plot(px, y, linewidth=1, label=f'{names[i]} %.3f' % ap[i, 0])  # plot(recall, precision)
    else:
        ax.plot(px, py, linewidth=1, color='grey')  # plot(recall, precision)

    ax.plot(px, py.mean(1), linewidth=3, color='blue', label='all classes %.3f mAP@0.5' % ap[:, 0].mean())
    ax.set_xlabel('Recall')
    ax.set_ylabel('Precision')
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
    fig.savefig(Path(save_dir) / 'precision_recall_curve.png', dpi=250)



In [ ]:
%cd "/kaggle/working/yolov5/"
!WANDB_MODE="online" python train.py --img 650 --batch 16 --epochs 35 --data /kaggle/working/smartathon.yaml --weights yolov5x.pt --cache

## Output in CSV

In [ ]:
%cd "/kaggle/input"
os.makedirs('/kaggle/working/smartathon/test', exist_ok = True)

In [ ]:
%cd "/kaggle/working/yolov5/"

In [ ]:
%cd /kaggle/working/yolov5
import torch
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/kaggle/working/yolov5/runs/train/exp/weights/best.pt', force_reload=True) 

In [ ]:
%cd /kaggle/input
for i,j in enumerate(test['image_path']):
    img = cv2.imread("../input/smartathon/dataset/images/" + j.split("/")[-1], cv2.IMREAD_UNCHANGED) 
    width = 960
    height = 540 # keep original height
    dim = (width, height)
    # resize image
    resized = cv2.resize(img, dim)
    cv2.imwrite("/kaggle/working/smartathon/resize_images/" + j.split("/")[-1],resized)

In [ ]:
test['image_path'] = "/kaggle/working/smartathon/resize_images/" + test['image_path']

In [ ]:
%cd "/kaggle/working"
final_output = pd.DataFrame(columns=["xmin", "ymin", "xmax", "ymax", "confidence", "class", "name", "image_path"])
for i,j in enumerate(test['image_path']):
    results = model(j,size=672,augment=True)
    output = results.pandas().xyxy[0]
    output['image_path'] = j.split("/")[-1]
    output = output[output['confidence'] >= 0.4]
    if output.shape[0] == 0:
        output = output.append(pd.DataFrame([[np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,j.split("/")[-1]]],columns=["xmin", "ymin", "xmax", "ymax", "confidence", "class", "name", "image_path"]),ignore_index=True)
    final_output = final_output.append(output)

In [ ]:
final_output = final_output.sort_values(['image_path','confidence'],ascending=False)

In [ ]:
### change columnsput format
final_output = final_output.reset_index(drop=True)
final_output = final_output[["class","image_path","name","xmax","xmin","ymax","ymin","confidence"]]

In [ ]:
final_output.shape

In [ ]:
final_output.to_csv("augment_inf_V1_0.4.csv",index=False)

In [ ]:
# %cd "/kaggle/working"
# # Image
# img = test['image_path'].iloc[7]

# # Inference
# results = model(img)

# results.pandas().xyxy[0].shape[0]

In [ ]:
# results.show()

In [ ]:
## Zip weights for further use
%cd /kaggle/working
!tar -zcvf str_split_weights.tar.gz /kaggle/working/yolov5/runs/train/exp/weights/